# K-MEANS SEGMENTING AND CLUSTERING VENUES IN TORONTO TO INSERT A COVID19 MEMORIAL OBELISK AS A POINT OF INTEREST

<h1>Applied Data Science Capstone by IBM/Coursera</h1>

<h3>Capstone Project - The Battle of Neighborhoods (2)</h3>

**Installing libraries for data analysis**

In [ ]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup

soup = BeautifulSoup(website_url,'lxml')
soup.prettify;

from sklearn.cluster import KMeans
#from sklearn.datasets import load_digits
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

In [ ]:
pip install geopy

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

**Data collection and Dataframes**<br>
Sources of data: <br>
List of postal codes of Canada, from wikipedia<br>
World Health Organization: Covid19 Dashboard<br>
FourSquare

In [ ]:
gc = pd.read_excel('Geospatial_Coordinates_1.xlsx')
gc

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

**Neighborhood Geolocalization and mapping**

In [ ]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(gc['Latitude'], gc['Longitude'], gc['Borough'], gc['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**The 100 important venues in Toronto**

In [ ]:
CLIENT_ID = 'FNRS21YKKOHZNCXE2IVQX3GCUSVYOUXZKLHTG1FZR2EK2C0K' # your Foursquare ID
CLIENT_SECRET = 'CG5K5YGHICCDTMTAAW441NHK5XPBHNR0FUKQI4BVROGIX4BE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Including Covid19 Memorial Obelisk geolocation, in Roselawn, Toronto - CA**

In [ ]:
Canada_Covid19 = pd.read_csv('WHO-COVID-19-global-data.csv')
Canada = Canada_Covid19.loc[Canada_Covid19['Country']=='Canada']
Canada = Canada.loc[Canada['Date_reported']=='2021-01-01']
Canada

In [ ]:
toronto_venues=pd.read_csv('toronto_vns.csv')

In [ ]:
toronto_venues.loc[len(toronto_venues.index)] = [1747,'Roselawn',43.704312,-79.409354,'Covid19 Memorial Obelisk ',43.704312,-79.409354,'Monument']

In [ ]:
toronto_venues.head()

In [ ]:
toronto_venues.tail()

In [ ]:
toronto_venues.shape

**Venues for each neighborhood**

In [ ]:
toronto_venues.groupby('Neighbourhood').count()

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

In [ ]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')

**Most common venues grouped by neighborhood**<br>

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

In [ ]:
neighbourhoods_venues_sorted.shape

**Clustering in map**<br>
5 clusters

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = gc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

In [ ]:
toronto_merged=toronto_merged.dropna()

In [ ]:
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

**Localization of the Toronto Covid19 Memorial Obelisk**

In [ ]:
display(toronto_venues.loc[1747])

**Determination of optimal number of clusters on venues**<br>
Elbow curve

In [ ]:
df_c = toronto_venues
df_c.shape

In [ ]:
X = df_c.loc[:,['Venue','Venue Latitude','Venue Longitude']]
X.head()

In [ ]:
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = df_c[['Venue Latitude']]
X_axis = df_c[['Venue Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]
# Visualize
plt.plot(K_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

**Optimal number of clusters = 3**

In [ ]:
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['cluster_label'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.df
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point
X.shape

**Graphic of venues segmentation around  the Covid19 Memorial Obelisk in Toronto** 

In [ ]:
#plt.figure(figsize=(10,8)) viridis
X.plot.scatter(x = 'Venue Latitude', y = 'Venue Longitude', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='blue', s=300, alpha=0.5)
plt.scatter(x = 43.7043,y = -79.4094, c='red', s=150, alpha=0.5)
plt.show()

**Centroids in blue color, venues grouped in yellow, green, and violet color, Covid19 Monumental Obelisk in red color**

In [ ]:
centers = kmeans.cluster_centers_
centers

In [ ]:
X.info()

In [ ]:
label0 = X.loc[X['cluster_label']==0]
label1 = X.loc[X['cluster_label']==1]
label2 = X.loc[X['cluster_label']==2]

In [ ]:
label0.shape

In [ ]:
label1.shape

In [ ]:
label2.shape

In [ ]:
label0.tail()

In [ ]:
label1.head()

In [ ]:
label2.head()

<h2>Globally, as of 9:38am CET, 6 January 2021, there have been 84.780.171 confirmed cases of COVID-19, including 1.853.525 deaths, reported to WHO.</h2>